In [67]:
import pandas as pd
import numpy as np
from weekly_predictions import run_weekly_predictions

In [68]:
# Data parameters
YEAR: int = 2024
WEEK: int = 3

# Wagering parameters
WAGER_SIZE: int = 20
CONFIDENCE_THRESHOLD: float = 0.75

In [69]:
df = run_weekly_predictions(YEAR, WEEK)

In [70]:
df.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,predicted_winner,win_probability
32,2024_03_NE_NYJ,2024,REG,3,2024-09-19,Thursday,20:15,NE,NaN,NYJ,...,00-0023459,Jacoby Brissett,Aaron Rodgers,Jerod Mayo,Robert Saleh,NaN,NYC01,MetLife Stadium,NE,0.581463
33,2024_03_NYG_CLE,2024,REG,3,2024-09-22,Sunday,13:00,NYG,NaN,CLE,...,00-0033537,Daniel Jones,Deshaun Watson,Brian Daboll,Kevin Stefanski,NaN,CLE00,FirstEnergy Stadium,CLE,0.781090
34,2024_03_CHI_IND,2024,REG,3,2024-09-22,Sunday,13:00,CHI,NaN,IND,...,00-0039164,Caleb Williams,Anthony Richardson,Matt Eberflus,Shane Steichen,NaN,IND00,Lucas Oil Stadium,IND,0.537212
35,2024_03_HOU_MIN,2024,REG,3,2024-09-22,Sunday,13:00,HOU,NaN,MIN,...,00-0034869,C.J. Stroud,Sam Darnold,DeMeco Ryans,Kevin O'Connell,NaN,MIN01,U.S. Bank Stadium,MIN,0.751295
36,2024_03_PHI_NO,2024,REG,3,2024-09-22,Sunday,13:00,PHI,NaN,NO,...,00-0031280,Jalen Hurts,Derek Carr,Nick Sirianni,Dennis Allen,NaN,NOR00,Mercedes-Benz Superdome,NO,0.929497


In [71]:
# Function to calculate implied probability from moneyline
def implied_prob(moneyline):
    if moneyline > 0:
        return 100 / (moneyline + 100)
    else:
        return -moneyline / (-moneyline + 100)


# Function to calculate potential profit from a moneyline bet
def calculate_profit(moneyline, wager):
    if moneyline > 0:
        return wager * (moneyline / 100)
    else:
        return wager * (100 / -moneyline)


# Function to calculate expected profit
def expected_profit(win_probability, moneyline, wager):
    potential_payout = calculate_profit(moneyline, wager)
    return (win_probability * potential_payout) - ((1 - win_probability) * wager)

In [72]:
# Calculate implied probabilities for home and away teams from the moneyline
df["away_implied_prob"] = df["away_moneyline"].apply(implied_prob)
df["home_implied_prob"] = df["home_moneyline"].apply(implied_prob)

# Compare implied probability with model's predicted probability for value bets
df["away_value_bet"] = df["win_probability"] < df["away_implied_prob"]
df["home_value_bet"] = df["win_probability"] > df["home_implied_prob"]

# Calculate potential profit for away and home teams
df["away_potential_profit"] = df["away_moneyline"].apply(
    lambda x: calculate_profit(x, WAGER_SIZE)
)
df["home_potential_profit"] = df["home_moneyline"].apply(
    lambda x: calculate_profit(x, WAGER_SIZE)
)

# Calculate the projected profit based on the predicted winner (use home or away moneyline based on the prediction)
df["projected_profit"] = np.where(
    df["predicted_winner"] == df["away_team"],
    df["away_potential_profit"],  # Profit if away team wins
    df["home_potential_profit"],  # Profit if home team wins
)

In [73]:
# Display the top rows with the projected profit
sorted_df = (
    df[
        [
            "away_team",
            "home_team",
            "predicted_winner",
            "win_probability",
            "away_moneyline",
            "home_moneyline",
            "projected_profit",
        ]
    ]
    .sort_values(by="projected_profit", ascending=False)
    .head(10)
)

In [74]:
# Filter for win probability above threshold and value bets
filtered_df = (
    df.loc[
        (df["win_probability"] > CONFIDENCE_THRESHOLD),
        [
            "projected_profit",
            "away_team",
            "home_team",
            "predicted_winner",
            "win_probability",
            "away_moneyline",
            "home_moneyline",
        ],
    ]
    .sort_values(by="projected_profit", ascending=False)
    .head(10)
)


display(filtered_df)

,projected_profit,away_team,home_team,predicted_winner,win_probability,away_moneyline,home_moneyline
36,24.000000,PHI,NO,NO,0.929497,-142.0,120.0
45,9.174312,KC,ATL,KC,0.841972,-218.0,180.0
46,7.194245,JAX,BUF,BUF,0.826018,225.0,-278.0
38,6.153846,DEN,TB,TB,0.897229,260.0,-325.0
33,5.882353,NYG,CLE,CLE,0.781090,270.0,-340.0
40,5.882353,CAR,LV,LV,0.851380,270.0,-340.0
35,NaN,HOU,MIN,MIN,0.751295,NaN,NaN


In [75]:
# Bets Taken So Far
TEAMS_BET = ["NO", "PHI", "NE", "KC", "BUF"]

# Game Rows
game_rows = df.loc[
    (df["away_team"].isin(TEAMS_BET) | df["home_team"].isin(TEAMS_BET)),
    [
        "projected_profit",
        "away_team",
        "home_team",
        "predicted_winner",
        "win_probability",
        "away_moneyline",
        "home_moneyline",
    ],
]

display(game_rows)

,projected_profit,away_team,home_team,predicted_winner,win_probability,away_moneyline,home_moneyline
32,49.000000,NE,NYJ,NE,0.581463,245.0,-305.0
36,24.000000,PHI,NO,NO,0.929497,-142.0,120.0
45,9.174312,KC,ATL,KC,0.841972,-218.0,180.0
46,7.194245,JAX,BUF,BUF,0.826018,225.0,-278.0
